### start up the EEG
make sure that the EEG is tested  
that the switch on the USB dongle is at GPOI6  
that the switch on the back of the EEG cap is in 'up' position  
that the latency times is set properly [windows](https://openbci.com/forum/index.php?p=/discussion/198/timing-marks-buffering-delays-etc) or [linux](https://askubuntu.com/questions/696593/reduce-request-latency-on-an-ftdi-ubs-to-rs-232-adapter)  
`echo 1 | sudo tee /sys/bus/usb-serial/devices/ttyUSB0/latency_timer`

Then open a terminal  
cd to ~/nf/openBCI_LSL  
then type:  
`python openbci_lsl.py -- stream`

followed by:

`/start`

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# make the connection to the EEG Net (EEG Data)
import pylsl
data_stream=pylsl.resolve_byprop("name", "openbci_eeg", timeout=5.0)
if data_stream:
    data_inlet=pylsl.stream_inlet(data_stream[0])
    stream_info = data_inlet.info()
    stream_Fs = stream_info.nominal_srate()
    stream_xml = stream_info.desc()
    chans_xml = stream_xml.child("channels")
    chan_xml_list = []
    ch = chans_xml.child("channel")
    while ch.name() == "channel":
        chan_xml_list.append(ch)
        ch = ch.next_sibling("channel")
    channel_names = [ch_xml.child_value("label") for ch_xml in chan_xml_list]
    data_inlet_dt = data_inlet.time_correction(timeout=5.0)
    sampling_freq = data_stream[0].nominal_srate()
    print('name = %s' % data_stream[0].name())
    print('sampling_freq = %d' % sampling_freq)
    print('channel_count = %d' % data_stream[0].channel_count())
    print('channel_format = %d' % data_stream[0].channel_format())
else:
    raise Exception('No Data Stream Found - Is your EEG Cap running?')

name = openbci_eeg
sampling_freq = 125
channel_count = 16
channel_format = 1


In [11]:
import sys
sys.path.append("/home/johan/nf/nfb")
sys.path.append("/home/johan/nf/nfb/pynfb")
import widgets
from widgets.signal_viewers import SignalViewer
from widgets.signal_viewers import RawSignalViewer

# w=SignalViewer(srate, channel_names, seconds_to_plot=5, overlap=True)
w=RawSignalViewer(sampling_freq, channel_names)
w.show()

In [8]:
from PyQt5 import QtWidgets
app = QtWidgets.QApplication(sys.argv)

In [3]:
channel_names

['Fp1',
 'Fp2',
 'C3',
 'C4',
 'T5',
 'T6',
 'O1',
 'O2',
 'F7',
 'F8',
 'F3',
 'F4',
 'T3',
 'T4',
 'P3',
 'P4']

In [4]:
### Collect the data
import time
import numpy as np

while data_inlet.samples_available(): data_inlet.pull_chunk() # clear buffer

    
collected_data = []
stop_time = time.time() + 25
while time.time() < stop_time:
    chunk_data, chunk_times = data_inlet.pull_chunk()
    if chunk_data: 
        collected_data.append(chunk_data)
        w.update(chunk_data)

data=np.concatenate(collected_data) # the data we wish to use for calibration

In [5]:
data.shape

(3126, 16)

In [6]:
# stuff that they import:

from copy import deepcopy

from PyQt5 import QtCore, QtGui, QtWidgets
import sys

from pynfb.inlets.montage import Montage
from pynfb.protocols import SelectSSDFilterWidget
from pynfb.protocols.signals_manager.band_selector import BandSelectorWidget
from pynfb.protocols.ssd.topomap_canvas import TopographicMapCanvas
from pynfb.protocols.ssd.topomap_selector_ica import ICADialog
from pynfb.widgets.rejections_editor import RejectionsWidget
from pynfb.widgets.spatial_filter_setup import SpatialFilterSetup
from pynfb.widgets.check_table import CheckTable
from pynfb.signals import DerivedSignal, BCISignal
from numpy import dot, concatenate, array
import numpy as np
from pynfb.widgets.bci_fit import BCIFitWidget

In [9]:
# how they run the command;
# the decomposition should be set to False!
result = ICADialog.get_rejection(data, channel_names, sampling_freq, decomposition=None)

Dropped 118 outliers
Creating RawArray with float64 data, n_channels=16, n_times=3008
    Range : 0 ... 3007 =      0.000 ...    24.056 secs
Ready.
Fitting ICA to data using 16 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 16
Computing Extended Infomax ICA
Fitting ICA took 2.5s.
ICA/CSP time elapsed = 2.5612850189208984s
Table drawing time elapsed = 3.1615853309631348s


<Figure size 432x288 with 0 Axes>

In [ ]:
channel_names

In [10]:
ica_rejection, filter, topography, ica_unmixing_matrix, bandpass, to_all = result

In [27]:
ica_unmixing_matrix

In [ ]:
# let's figure out how to use pudb to use 'remove' login. THis might actually work with the jupyerlab too